<a href="https://colab.research.google.com/github/IVMRanepa/courses.shift01/blob/main/Windows_Host_Security.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Практикум на Python 3 — Модуль 5. Безопасность хостов Windows**

**Задание 1 — Инвентаризация Windows-хостов (CMDB-стиль)**

**Цель:** сгенерировать список хостов, OS, роли и статус патчей.

In [ ]:
# task01_inventory.py
import random, json
hosts = []
for i in range(1,11):
    hosts.append({
        "hostname": f"win-host-{i}",
        "os": random.choice(["Windows Server 2019","Windows 10","Windows Server 2022"]),
        "role": random.choice(["DC","web","db","user"]),
        "patch_status": random.choice(["up-to-date","missing"])
    })
print(json.dumps(hosts, indent=2))

[
  {
    "hostname": "win-host-1",
    "os": "Windows 10",
    "role": "user",
    "patch_status": "missing"
  },
  {
    "hostname": "win-host-2",
    "os": "Windows Server 2022",
    "role": "user",
    "patch_status": "up-to-date"
  },
  {
    "hostname": "win-host-3",
    "os": "Windows Server 2019",
    "role": "user",
    "patch_status": "missing"
  },
  {
    "hostname": "win-host-4",
    "os": "Windows 10",
    "role": "db",
    "patch_status": "missing"
  },
  {
    "hostname": "win-host-5",
    "os": "Windows Server 2019",
    "role": "DC",
    "patch_status": "up-to-date"
  },
  {
    "hostname": "win-host-6",
    "os": "Windows Server 2019",
    "role": "DC",
    "patch_status": "missing"
  },
  {
    "hostname": "win-host-7",
    "os": "Windows 10",
    "role": "web",
    "patch_status": "missing"
  },
  {
    "hostname": "win-host-8",
    "os": "Windows 10",
    "role": "user",
    "patch_status": "missing"
  },
  {
    "hostname": "win-host-9",
    "os": "Windows Server

**Комментарий:** CMDB — основа, чтобы знать, что нужно защищать.

**Пример вывода:** JSON-список хостов с полями.

**Рекомендации:** синхронизировать CMDB с инвентаризацией (WMI, SCCM).

**Вывод:** инвентаризация — первый шаг к контролю безопасности.

**Задание 2 — Поиск уязвимых служб (симуляция)**

**Цель:** определить хосты с "опасными" слушающими портами (FTP, SMBv1)

In [ ]:
# task02_vuln_services.py
hosts_services = {
  "win-host-1":["RDP","SMBv1"], "win-host-2":["HTTP"], "win-host-3":["FTP","RDP"]
}
for host, svcs in hosts_services.items():
    risky = [s for s in svcs if s in ("SMBv1","FTP")]
    if risky:
        print(host, "risky services:", risky)

win-host-1 risky services: ['SMBv1']
win-host-3 risky services: ['FTP']


**Рекомендации:** отключать/переупаковывать SMBv1, использовать SFTP; применять host hardening.

**Вывод:** аудит сервисов помогает приоритизировать исправления.

**Задание 3 — Проверка политик паролей (симуляция GPO)**

**Цель:** смоделировать проверку минимальной длины пароля/истечения.

In [ ]:
# task03_password_policy_check.py
gpo = {"min_length":12,"max_age_days":90,"complexity":True}
users = {"alice": {"pwd_len":8,"pwd_age":30}, "bob":{"pwd_len":14,"pwd_age":120}}
for u,info in users.items():
    issues=[]
    if info["pwd_len"]<gpo["min_length"]: issues.append("short_pwd")
    if info["pwd_age"]>gpo["max_age_days"]: issues.append("expired")
    print(u, "issues:", issues or "OK")

alice issues: ['short_pwd']
bob issues: ['expired']


**Рекомендации:** применять GPO, MFA для админов, LAPS для локальных админов.

**Вывод:** автоматический аудит GPO ускоряет исправление проблем.

**Задание 4 — Проверка контроля учётных записей (UAC) в конфигурации**

**Цель:** убедиться, что UAC включён (симуляция).

In [ ]:
# task04_uac_check.py
hosts = {"win-host-1":{"UAC":True}, "win-host-2":{"UAC":False}}
for h,conf in hosts.items():
    if not conf["UAC"]:
        print(h,"UAC disabled — risk")
    else:
        print(h,"UAC enabled")

win-host-1 UAC enabled
win-host-2 UAC disabled — risk


**Рекомендации:** включать UAC, контролировать через GPO.

**Вывод:** UAC — простой но важный барьер.

**Задание 5 — Поиск SUID-аналогов (сервисов, запущенных от SYSTEM)**

**Цель:** найти процессы, запущенные от SYSTEM (симуляция).

In [ ]:
# task05_system_processes.py
processes = [
  {"proc":"svchost.exe","user":"SYSTEM"},
  {"proc":"notepad.exe","user":"user1"}
]
for p in processes:
    if p["user"]=="SYSTEM":
        print("High-priv proc:", p["proc"], "running as SYSTEM")

High-priv proc: svchost.exe running as SYSTEM


**Комментарий:** процессы от SYSTEM — важные точки контроля.

**Рекомендации:** минимизировать сервисы под SYSTEM, использовать Managed Service Accounts.

**Вывод:** учет процессов нужен для поиска потенциальных escalation точек.

**Задание 6 — Монитор реестра: детектирование изменений (имитация)**

**Цель:** показать принцип мониторинга ключевых реестров.

In [ ]:
# task06_registry_monitor.py
baseline = {"HKLM\\Software\\MyApp":"v1.0"}
current = {"HKLM\\Software\\MyApp":"v1.1"}
for k,v in current.items():
    if baseline.get(k)!=v:
        print("Registry changed:", k, baseline.get(k), "->", v)

Registry changed: HKLM\Software\MyApp v1.0 -> v1.1


**Рекомендации:** мониторить критичные ключи (Run, Services, PowerShell execution policy).

**Вывод:** мониторинг реестра помогает обнаружить persistence техники.